In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/MyDrive/RotoNet'

/content/drive/MyDrive/RotoNet


In [ ]:
!git clone https://github.com/yangchris11/samurai.git

Cloning into 'samurai'...
remote: Enumerating objects: 1007, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 1007 (delta 78), reused 41 (delta 41), pack-reused 885 (from 1)
Receiving objects: 100% (1007/1007), 376.22 MiB | 40.79 MiB/s, done.
Resolving deltas: 100% (238/238), done.


In [ ]:
%cd drive/MyDrive/RotoNet/samurai
!python scripts/demo.py --video_path data/input.mp4 --txt_path data/bbox.txt

In [ ]:
import cv2
import numpy as np
import os
from google.colab import files

video_path = '/content/drive/MyDrive/RotoNet/samurai/data/input.mp4'
mask_dir = '/content/drive/MyDrive/RotoNet/samurai/masks'
output_path = '/content/drive/MyDrive/RotoNet/samurai/data/mask.mp4'

cap = cv2.VideoCapture(video_path)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

for i in range(total_frames):
    ret, frame = cap.read()
    if not ret:
        break
    mask_path = os.path.join(mask_dir, f'frame{i:03d}_obj0.png')
    if not os.path.exists(mask_path):
        print(f'Mask not found for frame {i}, skipping...')
        continue

    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, (frame_width, frame_height))
    mask = mask // 255
    mask = np.repeat(mask[:, :, np.newaxis], 3, axis=2)
    object_only = frame * mask
    out.write(object_only.astype(np.uint8))

cap.release()
out.release()

files.download(output_path)


FileNotFoundError: Cannot find file: /content/drive/MyDrive/RotoNet/samurai/data/mask.mp4

In [ ]:
!git clone https://github.com/westgarthb/style-transfer-video-processor.git -q

In [ ]:
%cd '/content/drive/MyDrive/RotoNet/style-transfer-video-processor'

/content/drive/MyDrive/RotoNet/style-transfer-video-processor


In [ ]:
!pip -q install -r requirements.txt 2> /dev/null

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 146.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 98.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 MB 5.8 MB/s eta 0:00:00
  Installing build dependencies ... error


In [ ]:
from google.colab import files
uploaded_file = None

In [ ]:
uploaded_file = files.upload()

Saving mask.mp4 to mask.mp4


In [ ]:
class Config:
    ROOT_PATH = '.'
    FRAME_HEIGHT = 368
    CLEAR_INPUT_FRAME_CACHE = True
    INPUT_FPS = 20
    INPUT_VIDEO_NAME = 'input_vid.mov'
    if uploaded_file:
      INPUT_VIDEO_NAME = list(uploaded_file.keys())[0]
    INPUT_VIDEO_PATH = f'{ROOT_PATH}/{INPUT_VIDEO_NAME}'
    INPUT_FRAME_DIRECTORY = f'{ROOT_PATH}/input_frames'
    INPUT_FRAME_FILE = '{:0>4d}_frame.png'
    INPUT_FRAME_PATH = f'{INPUT_FRAME_DIRECTORY}/{INPUT_FRAME_FILE}'

    STYLE_REF_DIRECTORY = f'{ROOT_PATH}/style_ref'
    STYLE_SEQUENCE = [0, 1, 2]

    OUTPUT_FPS = 20
    OUTPUT_VIDEO_NAME = 'output_video.mp4'
    OUTPUT_VIDEO_PATH = f'{ROOT_PATH}/{OUTPUT_VIDEO_NAME}'
    OUTPUT_FRAME_DIRECTORY = f'{ROOT_PATH}/output_frames'
    OUTPUT_FRAME_FILE = '{:0>4d}_frame.png'
    OUTPUT_FRAME_PATH = f'{OUTPUT_FRAME_DIRECTORY}/{OUTPUT_FRAME_FILE}'

    MASK_FRAME_DIRECTORY = f'{ROOT_PATH}/masks'

    GHOST_FRAME_TRANSPARENCY = 0.1
    PRESERVE_COLORS = False

    TENSORFLOW_CACHE_DIRECTORY = f'{ROOT_PATH}/tensorflow_cache'
    TENSORFLOW_HUB_HANDLE = 'https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2'


In [ ]:
import os
import re

folder_path = '/content/drive/MyDrive/RotoNet/style-transfer-video-processor/masks'

pattern = re.compile(r'frame(\d+)_obj\d+\.png')

for filename in os.listdir(folder_path):
    match = pattern.match(filename)
    if match:
        num_str = match.group(1)
        new_name = f"{int(num_str):04d}_frame.png"
        old_path = os.path.join(folder_path, filename)
        new_path = os.path.join(folder_path, new_name)
        print(f"Rename: {filename} -> {new_name}")
        os.rename(old_path, new_path)


In [ ]:
from style_frames import StyleFrame
StyleFrame(Config).run()

Getting input frames
Getting style info
Getting output frames
Output frame: 0%
Output frame: 5%
Output frame: 11%
Output frame: 16%
Output frame: 21%
Output frame: 27%
Output frame: 32%
Output frame: 37%
Output frame: 43%
Output frame: 48%
Output frame: 53%
Output frame: 59%
Output frame: 64%
Output frame: 70%
Output frame: 75%
Output frame: 80%
Output frame: 86%
Output frame: 91%
Output frame: 96%
Saving video
Saving frame: 0%
Saving frame: 5%
Saving frame: 11%
Saving frame: 16%
Saving frame: 21%
Saving frame: 27%
Saving frame: 32%
Saving frame: 37%
Saving frame: 43%
Saving frame: 48%
Saving frame: 53%
Saving frame: 59%
Saving frame: 64%
Saving frame: 70%
Saving frame: 75%
Saving frame: 80%
Saving frame: 86%
Saving frame: 91%
Saving frame: 96%
Style transfer complete! Output at ./output_video.mp4


In [ ]:
files.download('output_video.mp4')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>